In [1]:
#Part 2
#ex 1
# Install MongoDB or connect to a web Mongo interface. Create MongoMusic
# database with 3 collections: bands, albums, songs.

In [2]:
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint
from datetime import datetime
import time

In [3]:
# connect to MongoDB
client = MongoClient('localhost:27017')
db=client.admin
# Issue the serverStatus command and print the results
serverStatusResult=db.command("serverStatus")
# pprint(serverStatusResult)

In [4]:
list_of_db = client.list_database_names()

In [5]:
list_of_db

['MongoMusic', 'admin', 'config', 'local']

In [6]:
db = client['MongoMusic']

In [7]:
bands_collection = db["bands"]
albums_collection = db["albums"]
songs_collection = db["songs"]
bands_info_collection = db["bands_info"]

In [8]:
# ex 2

In [9]:
bands_json = pd.read_json('bands.json', encoding='UTF-8').reset_index().to_json(orient='table', index=False)
albums_json = pd.read_json('albums.json', encoding='UTF-8').reset_index().to_json(orient='table', index=False)
songs_json = pd.read_json('songs.json', encoding='UTF-8').reset_index().to_json(orient='table', index=False)
bands_json = json.loads(bands_json)["data"]
albums_json = json.loads(albums_json)["data"]
songs_json = json.loads(songs_json)["data"]

In [10]:
bands_collection.drop()
albums_collection.drop()
songs_collection.drop()
db['all_rock_songs'].drop()
db['bands_info'].drop()

In [11]:
bands_collection.insert_many(bands_json)

In [12]:
albums_collection.insert_many(albums_json)
songs_collection.insert_many(songs_json)

In [13]:
albums_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'MongoMusic'), 'albums')

In [14]:
#ex 3

In [15]:
cursor = bands_collection.find( {"name":'Cake'})
for document in cursor:
    print(document)

In [16]:
def check_bandname_exists(band_name):
    cursor = bands_collection.find( {"name":band_name})
    for document in cursor:
        if document:
            return True
    return False

In [17]:
check_bandname_exists('Black Sabbath')

True

In [18]:
band_names_to_add = ['Space Buddha', 'Blondie', 'Sex Pistols', 'Abba', 'New Order', 'Nancy Sinatra', 'Dolly Parton', 'Television', 'Talking heads']

In [19]:
for band_name in band_names_to_add:
    print(band_name, ": ", check_bandname_exists(band_name))

Space Buddha :  False
Blondie :  False
Sex Pistols :  False
Abba :  False
New Order :  False
Nancy Sinatra :  False
Dolly Parton :  False
Television :  False
Talking heads :  False


In [20]:
for row in bands_collection.find({}):
    final_idx = row['id']#.reverse().limit(1)

In [21]:
band_rows_to_save = []
i = final_idx + 1
for band_name in band_names_to_add:
    band_row_to_save = {'name': band_name, 'id':i}
    band_rows_to_save.append(band_row_to_save)
    i+=1

In [22]:
bands_collection.insert_many(band_rows_to_save)

In [23]:
for band_name in band_names_to_add:
    print(band_name, ": ", check_bandname_exists(band_name))

Space Buddha :  True
Blondie :  True
Sex Pistols :  True
Abba :  True
New Order :  True
Nancy Sinatra :  True
Dolly Parton :  True
Television :  True
Talking heads :  True


In [24]:
import discogs_client
d = discogs_client.Client('music/1.0', user_token='mdgCpGzrCTLBRMsFWkKdWYAXwjRvPoxSxainTvPJ')

In [25]:
members = {}
artist_ids = {}
i = 0
for band_c in bands_collection.find({}):
    time.sleep(0.2)
    band_name = band_c["name"]
    result = d.search(band_name)
    for r in result:
        time.sleep(0.1)
        if isinstance(r, discogs_client.models.Artist):
            artist_ids[band_name] = r.data['id']
            members[band_name] = []
            for member in r.members:
                time.sleep(0.2)
                member_name = member.data['name']
                idx = member_name.find(" (")
                if idx != -1:
                    member_name = member_name[:idx]
                    
                members[band_name].append(member_name)
            members[band_name] = np.array(members[band_name])
            break

In [26]:
artist_ids

{'Aerosmith': 48424,
 'Metallica': 18839,
 'Def Leppard': 252443,
 'Iron Maiden': 251595,
 'Incognito': 763,
 'The Clash': 29890,
 'R.E.M.': 74500,
 'Gilberto Gil': 83724,
 'Jorge Ben': 30700,
 'Antônio Carlos Jobim': 27991,
 'Gerald Moore': 845919,
 'David Coverdale': 369909,
 'Santana': 30724,
 'Eric Clapton': 17827,
 'Kiss': 153073,
 'The Rolling Stones': 20991,
 'Ozzy Osbourne': 59770,
 'Emerson String Quartet': 848555,
 'Led Zeppelin': 34278,
 'Gustav Mahler': 239236,
 'O Terço': 1345178,
 'Marillion': 218108,
 'Michael Tilson Thomas & San Francisco Symphony': 253245,
 'Green Day': 251593,
 'Van Halen': 94066,
 "The King's Singers": 739941,
 'English Concert & Trevor Pinnock': 837588,
 'Djavan': 23475,
 'Nash Ensemble': 851241,
 'Sergei Prokofiev & Yuri Temirkanov': 1880406,
 'Chico Science & Nação Zumbi': 478641,
 'Luciano Pavarotti': 55683,
 'Black Sabbath': 144998,
 'Yehudi Menuhin': 532086,
 'Milton Nascimento': 83723,
 'U2': 6520,
 'Toquinho & Vinícius': 961600,
 'Audioslave'

In [28]:
members

{'Aerosmith': array(['Joe Perry', 'Joey Kramer', 'Jimmy Crespo', 'Tom Hamilton',
        'Brad Whitford', 'Rick Dufay', 'Ray Tabano', 'Steven Tallarico'],
       dtype='<U16'),
 'Metallica': array(['Kirk Hammett', 'Lars Ulrich', 'Dave Mustaine', 'James Hetfield',
        'Cliff Burton', 'Jason Newsted', 'Robert Trujillo', 'Ron McGovney'],
       dtype='<U15'),
 'Def Leppard': array(['Phil Collen', 'Joe Elliott', 'Rick Savage', 'Pete Willis',
        'Steve Clark', 'Rick Allen', 'Vivian Campbell'], dtype='<U15'),
 'Iron Maiden': array(['Steve Harris', 'Nicko McBrain', 'Bruce Dickinson', 'Janick Gers',
        'Dave Murray', 'Adrian Smith', 'Clive Burr', 'Dennis Stratton',
        'Paul Mario Day', 'Doug Sampson', 'Ron "Rebel" Matthews',
        'Tony Parsons', 'Tony Hustings-Moore', 'Paul Cairns', 'Bob Sawyer',
        'Terry Wapram', 'Paul Todd', 'Dennis Willcock', 'Terry Rance',
        'Paul Michael Andrews', 'Barry Graham Purkis', 'Bayley Cooke'],
       dtype='<U20'),
 'Incognito':

In [29]:
countries = {'Space Buddha': 'Israel', 'Blondie': 'New York, New York, USA', 'Sex Pistols': 'London, Great Britian',
            'Abba': 'Sweden', 'New Order': 'Manchester, Great Britian', 'Nancy Sinatra': 'Jersey City, New Jersey, USA',
             'Dolly Parton': 'Sevier County, Tennessee, USA',  'Television': 'New York, New York, USA',
             'Talking heads': 'New York, New York, USA'}

In [31]:
list_of_records = []
for band_name in members.keys():
    data_dict = {}
    cursor = bands_collection.find({'name':band_name})
    data_dict['band'] = cursor[0]['id']
    data_dict['band_name'] = band_name
    if band_name in countries.keys():
        data_dict['place_of_origin'] = countries[band_name]
    band_members = members[band_name]
    members_info = []
    for band_member in band_members:
        band_member_dict = {}
        band_member_dict['name'] = band_member
        time.sleep(0.3)
        result = d.search(band_member)
        time.sleep(0.2)
        for r in result:
            time.sleep(0.2)
            if isinstance(r, discogs_client.models.Artist):
                member_id = r.data['id']
                time.sleep(0.3)
                other_bands = len(d.artist(member_id).groups)>1
                time.sleep(0.3)
                band_member_dict['other_bands'] = other_bands
                break
        members_info.append(band_member_dict)
    data_dict['members'] = members_info
    list_of_records.append(data_dict)

In [32]:
bands_info_collection.insert_many(list_of_records)

In [33]:
pprint(bands_info_collection)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'MongoMusic'), 'bands_info')


In [34]:
cursor = bands_info_collection.find({'band_name':'Space Buddha'})
cursor[0]

{'_id': ObjectId('61ecd50b27edbf3720f67ff9'),
 'band': 165,
 'band_name': 'Space Buddha',
 'place_of_origin': 'Israel',
 'members': [{'name': 'Eliad Grundland', 'other_bands': True},
  {'name': 'Yalon Adot', 'other_bands': True}]}

In [35]:
#ex 4

In [36]:
# select randomly bands which albums will be added
albums_bands = np.random.choice(band_names_to_add, 15)

In [37]:
albums_bands

array(['Television', 'Abba', 'Talking heads', 'Talking heads', 'Abba',
       'Space Buddha', 'Space Buddha', 'Nancy Sinatra', 'Television',
       'Blondie', 'Television', 'Abba', 'Space Buddha', 'Dolly Parton',
       'Talking heads'], dtype='<U13')

In [38]:
album_objects_dict = {}
albums_list = []
for row in albums_collection.find({}):
    final_idx = row['id']
final_idx += 1
for album_band in albums_bands:
    artist_id = artist_ids[album_band]
    time.sleep(0.2)
    releases = d.artist(artist_id).releases
    time.sleep(0.3)
    
    for release in releases:
        time.sleep(0.3)
        if isinstance(release, discogs_client.models.Master):
            if release not in album_objects_dict.values():#album of author not playlist
                album_dict = {}
                cursor = bands_collection.find({'name':album_band})
                album_dict['band'] = cursor[0]['id']
                album_dict['id'] = final_idx
                final_idx += 1
                album_dict['genre'] = release.genres
                album_dict['title'] = release.data['title']
                time.sleep(0.2)
                albums_list.append(album_dict)
                album_objects_dict[(album_dict['band'], final_idx)] = release
                break
            else:
                continue
    
    

In [39]:
album_objects_dict

{(172, 517): <Master 212377 'Little Johnny Jewel'>,
 (168, 518): <Master 1943 'Ring Ring (Bara Du Slog En Signal)'>,
 (173,
  519): <Master 39448 "Uh-oh, Love Comes To Town / I Wish You Wouldn't Say That">,
 (173, 520): <Master 54381 'Love Goes To Building On Fire'>,
 (168, 521): <Master 14485 'Honey Honey'>,
 (165, 522): <Master 1439107 'Reality Check'>,
 (165, 523): <Master 1111023 'Goa-Head Volume 16'>,
 (170,
  524): <Master 555009 'Cuff Links And A Tie Clip / Not Just Your Friend'>,
 (172, 525): <Master 6202 'Marquee Moon'>,
 (166, 526): <Master 365554 'In The Flesh'>,
 (172, 527): <Master 1816634 'Venus'>,
 (168, 528): <Master 4583 'So Long'>,
 (165, 529): <Master 891635 'Chillum 4 - The Ultimate Tribal Ambient Journey'>,
 (171, 530): <Master 1168847 'Puppy Love'>,
 (173, 531): <Master 39365 'Talking Heads: 77'>}

In [40]:
len(albums_list)

15

In [41]:
albums_collection.insert_many(albums_list)

In [42]:
for row in albums_collection.find({}):
    final_idx = row['id']
for row in albums_collection.find({}):
    if row['id']> final_idx-10:
        print(row['title'])

Reality Check
Goa-Head Volume 16
Cuff Links And A Tie Clip / Not Just Your Friend
Marquee Moon
In The Flesh
Venus
So Long
Chillum 4 - The Ultimate Tribal Ambient Journey
Puppy Love
Talking Heads: 77


In [43]:
# ex 5

In [44]:
(album_objects_dict.keys())

dict_keys([(172, 517), (168, 518), (173, 519), (173, 520), (168, 521), (165, 522), (165, 523), (170, 524), (172, 525), (166, 526), (172, 527), (168, 528), (165, 529), (171, 530), (173, 531)])

In [46]:
album_objects_dict[(172, 517)].tracklist

[<Track 'A' 'Little Johnny Jewel - Part One'>,
 <Track 'B' 'Little Johnny Jewel - Part Two'>]

In [47]:
def random_price():
    return np.round(np.random.random()*4+1, 2)



In [48]:
songs_list = []
for band_id, album_id in album_objects_dict.keys():
    album_object = album_objects_dict[(band_id, album_id)]
    tracklist = album_object.data['tracklist']
    for track in tracklist:
        song_dict = {}
        song_dict['band'] = band_id
        song_dict['album'] = album_id
        song_dict['price'] = random_price()
        song_dict['title'] = track['title']
        duration = track['duration']
        song_dict['length'] = track['duration']
        songs_list.append(song_dict)

In [49]:
len(songs_list)

78

In [50]:
songs_collection.insert_many(songs_list)

In [51]:
# ex6 

In [52]:
db.list_collection_names()

['albums', 'songs', 'bands', 'bands_info']

In [53]:
for collection in db.list_collections():
    print(collection)

{'name': 'albums', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b';#\x9b\xe5\x10\x08D\xd8\xb4\xf5\xf5\xa4\x80b\xbc4', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'songs', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'J\x1dk\\\x96\xa5Ms\xb3ax,h\xc3\x8d\x90', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'bands', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'rL\xa8:\xcdmBS\x81\xbd\x03<\xf4-z\x96', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'bands_info', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xd2ShJ\xa1YIh\xbdu\x83\xca\xe3\x13_|', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}


In [54]:
# ex 7 

In [55]:
for band_name in band_names_to_add:
    bands_cursor = bands_collection.find({'name':band_name})
    bands_cursor_list = list(bands_cursor)
    assert len(bands_cursor_list) == 1
    band_id = bands_cursor_list[0]['id']
    albums_cursor = albums_collection.find( {"band":band_id})
    i = len(list(albums_cursor))
    bands_collection.update_one({"id": band_id}, { "$set":{"noAlbums": i}})

In [56]:
cursor = list(bands_collection.find( {}))
for c in cursor[-12:]:
    print(c)

{'_id': ObjectId('61ecb6b627edbf3720f6742a'), 'index': 161, 'id': 162, 'name': 'UB40', 'noAlbums': 1}
{'_id': ObjectId('61ecb6b627edbf3720f6742b'), 'index': 162, 'id': 163, 'name': 'Edo de Waart & San Francisco Symphony', 'noAlbums': 1}
{'_id': ObjectId('61ecb6b627edbf3720f6742c'), 'index': 163, 'id': 164, 'name': 'R.E.M. Feat. Kate Pearson', 'noAlbums': 1}
{'_id': ObjectId('61ecb6bd27edbf3720f67f60'), 'name': 'Space Buddha', 'id': 165, 'noAlbums': 3}
{'_id': ObjectId('61ecb6bd27edbf3720f67f61'), 'name': 'Blondie', 'id': 166, 'noAlbums': 1}
{'_id': ObjectId('61ecb6bd27edbf3720f67f62'), 'name': 'Sex Pistols', 'id': 167, 'noAlbums': 0}
{'_id': ObjectId('61ecb6bd27edbf3720f67f63'), 'name': 'Abba', 'id': 168, 'noAlbums': 3}
{'_id': ObjectId('61ecb6bd27edbf3720f67f64'), 'name': 'New Order', 'id': 169, 'noAlbums': 0}
{'_id': ObjectId('61ecb6bd27edbf3720f67f65'), 'name': 'Nancy Sinatra', 'id': 170, 'noAlbums': 1}
{'_id': ObjectId('61ecb6bd27edbf3720f67f66'), 'name': 'Dolly Parton', 'id': 171,

In [57]:
# ex8 

In [58]:
all_bands_names_list = []
all_bands_name_cursor = bands_collection.find({})
for one_band in all_bands_name_cursor:
    band_id = one_band['id']
    albums_cursor = albums_collection.find( {"band":band_id})
    i = len(list(albums_cursor))
    bands_collection.update_one({"id": band_id}, { "$set":{"noAlbums": i}})

In [59]:
# ex 9 

In [60]:
curs = bands_info_collection.find({})
band_ids = []
for c in curs:
    for member in c['members']:
        if "John" in member['name']:
            band_ids.append(c['band'])
band_ids = list(set(band_ids))

In [61]:
band_ids

[6,
 136,
 139,
 14,
 17,
 21,
 151,
 153,
 26,
 158,
 31,
 35,
 163,
 167,
 45,
 52,
 54,
 66,
 80,
 81,
 82,
 83,
 92,
 95,
 101,
 104,
 105,
 112,
 120]

In [62]:
for band_id in band_ids:
    not_john_query = {"id":  band_id} 
    bands_collection.delete_many(not_john_query)
    not_john_query = {"band":  band_id}
    bands_info_collection.delete_many(not_john_query)
    albums_collection.delete_many(not_john_query)
    songs_collection.delete_many(not_john_query)

In [63]:
print(len(list(bands_collection.find({'id': 167}))))
print(len(list(bands_info_collection.find({'band': 167}))))
print(len(list(albums_collection.find({'band': 167}))))
print(len(list(songs_collection.find({'band': 167}))))

0
0
0
0


In [77]:
# ex 10

In [64]:
cursor = list(albums_collection.find( {'genre': 'Rock'}))
all_rock_songs_collection = db['all_rock_songs']
for c in cursor:
    band_id = c['band']
    album_id = c['id']
    band_cursor = list(bands_collection.find({'id': band_id}))
    assert len(band_cursor) == 1
    band_name = band_cursor[0]['name']
    song_cursor = songs_collection.find({'id': album_id})
    for sc in song_cursor:
        song_dict = {'title': sc['title'], 'band': band_id, 'album': album_id}
        all_rock_songs_collection.insert_one(song_dict)

In [65]:
curs = all_rock_songs_collection.find({})
for c in curs:
    print(c)
    break

{'_id': ObjectId('61ed3fd727edbf3720f6805f'), 'title': 'Walk On Water', 'band': 1, 'album': 1}


In [66]:
# ex 11

In [67]:
agg_result = albums_collection.aggregate([{
    "$group" : 
        {"_id" : "$genre", 
         "num_albums" : {"$sum" : 1}
         }}
    ])
# agg_result.find({"num_albums": {"$gt": 7}})
for i in agg_result:
    if i["num_albums"] >= 7:
        print(i['_id'])

Jazz
Alternative & Punk
Latin
Classical
Rock
Metal
['Rock']


In [68]:
# ex 12

In [69]:
agg_result = albums_collection.aggregate([{
    "$group" : 
        {"_id" : "$band", 
         "num_albums" : {"$sum" : 1},
         }}])

max_no = 0
agg_result_list = list(agg_result)
found = False
i = 0
while not found:
    agg_r = agg_result_list[i]
    i += 1
    if agg_r['num_albums']>max_no:
        max_no = agg_r['num_albums']
        band_id = agg_r["_id"]
    if i == len(agg_result_list):
        band_name = list(bands_collection.find({"id": band_id}))
        assert len(band_name) == 1
        band_name = band_name[0]
        print("no_albums: ", max_no)
        print("band_name: ", band_name["name"])
        found = True
        
        


no_albums:  19
band_name:  Iron Maiden


In [70]:
def get_time_seconds(time_str):
    date_time = datetime.strptime(time_str, '%M:%S')
    a_timedelta = date_time - datetime(1900, 1, 1)
    return a_timedelta.total_seconds()

In [71]:
# ex 13
min_len = 1e6
max_len = 0
song = None
song_list = list(songs_collection.find({}))
min_found = False
max_found = False
i = 0
while not min_found and not max_found and i<len(song_list):
# for song in songs_collection.find({}):
    length = song_list[i]['length']
    if length:
        if isinstance(length, str):
            length = get_time_seconds(length)
        if length> max_len:
            max_len = length
            max_song = song_list[i]
        if length<min_len:
            min_len = length
            min_song = song_list[i]
    i += 1

In [72]:
max_song

{'_id': ObjectId('61ecb6b727edbf3720f678d5'),
 'index': 918,
 'id': 919,
 'band': 59,
 'album': 106,
 'price': None,
 'title': "Space Truckin'",
 'length': 1196}

In [73]:
min_song

{'_id': ObjectId('61ecb6b727edbf3720f678b6'),
 'index': 887,
 'id': 888,
 'band': 72,
 'album': 103,
 'price': None,
 'title': 'É Uma Partida De Futebol',
 'length': 1}

In [74]:
# ex 14

In [75]:
# ex 13
avg_len = 1e6
max_len = 0
song = None
albums_curs = albums_collection.find({"genre": "Jazz"})
for one_album in albums_curs:
    songs_curs = songs_collection.find({"album":one_album["id"]})
    sum_length = 0
    n_songs = 0
    for song in songs_curs:
        n_songs += 1
        length = song['length']
        if length:
            if isinstance(length, str):
                length = get_time_seconds(length)
            sum_length += length
    avg_length = sum_length/n_songs
        
    print("Album name: ", one_album['title'], "avg_length: ", avg_length)


Album name:  Blue Moods avg_length:  340.0769230769231
Album name:  Quanta Gente Veio ver--Bônus De Carnaval avg_length:  305.3333333333333
Album name:  Quiet Songs avg_length:  277.0
Album name:  Worlds avg_length:  267.0
Album name:  Warner 25 Anos avg_length:  207.71428571428572
Album name:  The Essential Miles Davis [Disc 2] avg_length:  439.9
Album name:  Heart of the Night avg_length:  291.5833333333333
Album name:  Outbreak avg_length:  382.22222222222223
Album name:  Morning Dance avg_length:  275.22222222222223
Album name:  Miles Ahead avg_length:  221.8


In [76]:
# ex 15 

In [77]:
old_country_names_map = {"Macedonia": "Northern Macedonia", "Czech Republic": "Czechia"
                         , "USSR": "Russian Federation", "Great Britian":"Great Britain"}
resitrcted_countries = ["Yugoslavia"]

In [79]:
bands_info_curs = bands_info_collection.find({})
for i in bands_info_curs:
    if 'place_of_origin' in i.keys():
        current_country =  i['place_of_origin'].split(', ')
        band_id = i['band']
        if current_country[-1] in old_country_names_map.keys():
            print(current_country[-1])
            current_country[0] = old_country_names_map[current_country[-1]]
            bands_info_collection.update_one({"band": band_id}, { "$set":{'place_of_origin': ', '.join(current_country)}})
        elif current_country[0] in resitrcted_countries:
            print(current_country, "in restricted words")

Great Britian


In [93]:
# ex 16

In [82]:
bands_c

{'_id': ObjectId('61ecd50b27edbf3720f67f69'),
 'band': 1,
 'band_name': 'Aerosmith',
 'members': [{'name': 'Joe Perry', 'other_bands': True},
  {'name': 'Joey Kramer', 'other_bands': True},
  {'name': 'Jimmy Crespo', 'other_bands': True},
  {'name': 'Tom Hamilton', 'other_bands': True},
  {'name': 'Brad Whitford', 'other_bands': True},
  {'name': 'Rick Dufay', 'other_bands': True},
  {'name': 'Ray Tabano', 'other_bands': False},
  {'name': 'Steven Tallarico', 'other_bands': True}]}

In [93]:
all_people = {}
for bands_c in bands_info_collection.find({}):
    
    members = bands_c['members']
    band_id = bands_c['band']
    albums_c = albums_collection.find({'band': band_id})
    no_albums = len(list(albums_c))
    if members:
        for mem in members:
            author_name = mem['name']
            
            if author_name in all_people.keys():
                all_people[author_name] += no_albums
            else:  
                all_people[author_name] = no_albums

    else:
        author_name = mem['name'] = bands_c['band_name']
        if author_name in all_people.keys():
            all_people[author_name] += no_albums
        else:   
            all_people[author_name] = no_albums
            
for person in all_people.keys():
    print(person, all_people[person])

Joe Perry 1
Joey Kramer 1
Jimmy Crespo 1
Tom Hamilton 1
Brad Whitford 1
Rick Dufay 1
Ray Tabano 1
Steven Tallarico 1
Kirk Hammett 7
Lars Ulrich 7
Dave Mustaine 7
James Hetfield 7
Cliff Burton 7
Jason Newsted 7
Robert Trujillo 7
Ron McGovney 7
Phil Collen 1
Joe Elliott 1
Rick Savage 1
Pete Willis 1
Steve Clark 1
Rick Allen 1
Vivian Campbell 1
Steve Harris 19
Nicko McBrain 19
Bruce Dickinson 19
Janick Gers 19
Dave Murray 19
Adrian Smith 19
Clive Burr 19
Dennis Stratton 19
Paul Mario Day 19
Doug Sampson 19
Ron "Rebel" Matthews 19
Tony Parsons 19
Tony Hustings-Moore 19
Paul Cairns 19
Bob Sawyer 19
Terry Wapram 19
Paul Todd 19
Dennis Willcock 19
Terry Rance 19
Paul Michael Andrews 19
Barry Graham Purkis 19
Bayley Cooke 19
Jocelyn Brown 1
Tony Momrelle 1
Daniel Maunick 1
Kelli Sae 1
Joy Rose 1
Mark Anthoni 1
Jean-Paul Maunick 1
Stuart Zender 1
Linda Muriel 1
Ray Carless 1
Maysa Leak 1
Joy Malcolm 1
Paul "Tubbs" Williams 1
Peter Hinds 1
Tony Remy 1
Nomvula Malinga 1
Matt Cooper 1
Richard Bull

Paul Tobias 2
Bryan Mantia 2
Melissa Reese 2
Michael McKagan 2
Darren Arthur Reed 2
Ole Beich 2
Rob Gardner 2
Page & Plant 1
Aaron Goldberg 1
Caetano Veloso 2
Eddie Vedder 4
Mike McCready 4
Matt Cameron 5
Matt Chamberlain 4
Jack Irons 4
Jeff Ament 4
Stone Gossard 4
David Abbruzzese 4
Dave Krusen 4
Raul Seixas 1
Paula Frazer 3
Jim Martin 3
Mike Patton 3
Billy Gould 3
Dean Menta 3
Mike Bordin 3
Desmond Shea 3
Trey Spruance 3
Roddy Bottum 3
Courtney Love 3
Chuck Mosley 3
Jon Hudson 3
Mark Bowen 3
Jake Smith 3
Craig Shell 3
Joe Callahan 3
Walter O'Brien 3
Mark Stewart 3
Marcos Valle 1
Jimi Hendrix 1
Ton Koopman 1
Elis Regina 1
Larry Wallis 1
Eddie Clarke 1
Pete Gill 1
Brian Robertson 1
Phil Campbell 1
Phil Taylor 1
Lucas Fox 1
Michael Burston 1
Ian Fraser Kilmister 1
Micael Kiriakos Delaoglou 1
Lulu Santos 2
Paul D'Ianno 1
Philharmonia Orchestra & Sir Neville Marriner 1
Jon Auer 1
Ken Stringfellow 1
Joe Skyward 1
Matt Harris 1
Brian Young 1
Mike Musburger 1
Dave Fox 1
Darius Minwalla 1
Fra